# Census Data for San Antonio Districts

In [1]:
import pandas as pd
import numpy as np
import os
import re
import math

## District Data

This sheet tells which district each census tract aligns with and what percentage of the district is included. District 0 are census tracts outside of San Antonio.

In [2]:
dist = pd.read_excel('CMO_CDTractsPrecincts_220406.xlsx')
dist = dist[['Council District',
             'Tract 2020 ID\n(Use with 2020 Decennial Census)',
             'Percentage of Tract 2020 in Council District']]
dist.columns = ['council','tract','percent']
dist = dist.iloc[1:].reset_index(drop=True)
dist.tract = dist.tract.astype(str)
dist.council = np.where(dist.council == 'Outside CoSA', 0, dist.council)
dist.head()

,council,tract,percent
0,1,110100,0.999534
1,1,110300,0.499153
2,1,110500,0.000150
3,1,110600,0.002611
4,1,110700,0.953113


## Census Data

This is all the data pulled from the census. 

In [6]:
filename = 'ACSDP5Y2021.DP04-Data.csv'
filename2 = 'ACSDP5Y2021.DP04-Column-Metadata.csv'

In [149]:
dataall = pd.read_csv(filename)
dataall.columns = dataall.columns.str.lower()

dataall = dataall.iloc[1:].reset_index(drop=True)
dataall = dataall.replace('(X)', np.nan).replace('-',np.nan).replace('\*+',np.nan,regex=True)
# dataall = dataall.dropna(axis=1, how='all')

dataall.geo_id = dataall.geo_id.str[-6:]
dataall.geo_id = dataall.geo_id.astype(str)
dataall = dataall.drop(columns='name')

to_fix = []
for col in dataall.columns:
    bad_num = dataall[col].astype(str).str.contains('+',regex=False).sum()
    if bad_num > 0:
        to_fix.append(col)
for col in to_fix:
    dataall[col] = dataall[col].astype(str)\
    .str.replace('+','',regex=False)\
    .str.replace(',','').astype(float)

dataall.head()

,geo_id,dp04_0001e,dp04_0001ea,dp04_0001m,dp04_0001ma,dp04_0002e,dp04_0002m,dp04_0002ma,dp04_0002ea,dp04_0003e,...,dp04_0141pea,dp04_0142pe,dp04_0142pm,dp04_0142pma,dp04_0142pea,dp04_0143pe,dp04_0143pm,dp04_0143pma,dp04_0143pea,unnamed: 1146
0,110100,2958,NaN,272,NaN,1895,286,NaN,NaN,1063,...,NaN,31.1,11.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,110300,1888,NaN,265,NaN,1393,251,NaN,NaN,495,...,NaN,40.4,14.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,110500,1025,NaN,117,NaN,921,133,NaN,NaN,104,...,NaN,41.4,11.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,110600,1637,NaN,255,NaN,1551,269,NaN,NaN,86,...,NaN,51.9,13.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,110700,661,NaN,116,NaN,513,104,NaN,NaN,148,...,NaN,43.2,18.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Labels

These are the labels for the census data.

In [150]:
labels = pd.read_csv(filename2)
labels.columns = labels.columns.str.lower().str.replace('\W+','_', regex=True)
labels.column_name = labels.column_name.str.lower()

labels = labels [labels.label.str.startswith('Estimate')]
labels = labels [~labels.label.str.contains('ratio')]

groups = labels.label.str.split('!', expand=True)[2].unique()
groups

array(['HOUSING OCCUPANCY', 'UNITS IN STRUCTURE', 'YEAR STRUCTURE BUILT',
       'ROOMS', 'BEDROOMS', 'HOUSING TENURE',
       'YEAR HOUSEHOLDER MOVED INTO UNIT', 'VEHICLES AVAILABLE',
       'HOUSE HEATING FUEL', 'SELECTED CHARACTERISTICS',
       'OCCUPANTS PER ROOM', 'VALUE', 'MORTGAGE STATUS',
       'SELECTED MONTHLY OWNER COSTS (SMOC)',
       'SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME (SMOCAPI)',
       'GROSS RENT',
       'GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI)'],
      dtype=object)

## Functions to combine everything

In [151]:
def cal_counts(label):
    #pull out estimate and moa
    name_est = label + 'e'
    name_moa = label + 'm'

    #find the corresponding title to census label
    string = labels [labels.column_name == name_est].label.iloc[0]
#     label_title = re.findall('!!([\w\s]+)$',string)[0]
    if not string.split('!!')[3:]:
        label_title = string.split('!!')[-1]
    else:
        label_title = '!!'.join(string.split('!!')[3:])

    #isolate just one indicator
    data = dataall[['geo_id', name_est, name_moa]].copy()

    #join districts and indicator together
    df = dist.merge(data, how='inner', right_on='geo_id', left_on='tract')
    df = df.replace(np.nan, 0)

    #get count percentages for each census tract per district
    df['tru_count'] = df[name_est].astype(float) * df.percent
    df['tru_error'] = df[name_moa].astype(float) * df.percent

    #for each council district, sum up counts, and calculate new moe
    council_counts = []
    total_counts = df.tru_count.sum()

    for x in df.council.unique():
        subset = df [df.council == x]

        count = round(subset.tru_count.sum(), 2)
        count_perc = round(count / total_counts, 2)

        error = round((subset.tru_error ** 2).sum()**.5, 2)
        error_perc = round(math.sqrt(sum((subset.tru_error / 1.645)**2)) / count, 2)

        council_counts.append([x, count, count_perc, error, error_perc])

    #convert to dataframe and format
    dff = pd.DataFrame(council_counts).sort_values(0)
    dff.columns = [label_title, 'counts', 'count_perc', 'moe', 'moe_perc']
    dff = dff.set_index(label_title, drop=True)
#     print(dff)

    return dff

In [152]:
def cal_district_numbers(labels_check):
    district_totals = pd.DataFrame(np.arange(11))
    totals = []

    for label in labels_check:
        dff = cal_counts(label)
        name = dff.index.name
        dff.columns = [name + ' counts', name + ' counts percent', name + ' moe', name + ' moa percent']
        dff = dff.reset_index(drop=True)

        district_totals = pd.concat([district_totals,dff],axis=1)

    district_totals = district_totals.drop(columns=0)
    return district_totals.T

## Now calculate! 

### Total Housing

In [153]:
grouping = groups[0]

current_labels = labels [labels.label.str.contains(grouping)]
current_labels = current_labels.iloc[:1]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)

Estimate!!HOUSING OCCUPANCY!!Total housing units


In [154]:
final0 = cal_district_numbers(labels_check)
final0

,0,1,2,3,4,5,6,7,8,9,10
Total housing units counts,188661.64,57123.02,57254.27,53818.39,43705.62,45540.96,61319.53,61352.55,77166.25,69283.60,62837.16
Total housing units counts percent,0.24,0.07,0.07,0.07,0.06,0.06,0.08,0.08,0.10,0.09,0.08
Total housing units moe,2501.64,1317.08,1360.16,1189.09,939.71,1163.11,1410.20,1362.88,1474.12,1445.48,1184.30
Total housing units moa percent,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.01,0.01,0.01


### Housing units

In [155]:
grouping = groups[0]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
current_labels = current_labels.iloc[1:]
# current_labels = current_labels.iloc[:-13]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)

HOUSING OCCUPANCY

Estimate!!HOUSING OCCUPANCY!!Total housing units!!Occupied housing units
Estimate!!HOUSING OCCUPANCY!!Total housing units!!Vacant housing units
Estimate!!HOUSING OCCUPANCY!!Total housing units!!Homeowner vacancy rate
Estimate!!HOUSING OCCUPANCY!!Total housing units!!Rental vacancy rate


In [156]:
final1 = cal_district_numbers(labels_check)
final1

,0,1,2,3,4,5,6,7,8,9,10
Occupied housing units counts,177329.79,49774.31,51222.14,49100.15,40884.53,40961.91,57962.61,56476.88,69755.57,64733.36,58922.75
Occupied housing units counts percent,0.25,0.07,0.07,0.07,0.06,0.06,0.08,0.08,0.10,0.09,0.08
Occupied housing units moe,2512.64,1355.11,1382.56,1229.05,990.80,1169.42,1486.44,1416.77,1552.83,1487.35,1194.84
Occupied housing units moa percent,0.01,0.02,0.02,0.02,0.01,0.02,0.02,0.02,0.01,0.01,0.01
Vacant housing units counts,11331.85,7348.71,6032.14,4718.24,2821.09,4579.05,3356.92,4875.67,7410.68,4550.23,3914.41
Vacant housing units counts percent,0.19,0.12,0.10,0.08,0.05,0.08,0.06,0.08,0.12,0.07,0.06
Vacant housing units moe,965.12,652.99,567.36,505.92,403.10,469.36,581.22,545.72,766.29,585.23,538.89
Vacant housing units moa percent,0.05,0.05,0.06,0.07,0.09,0.06,0.11,0.07,0.06,0.08,0.08
Homeowner vacancy rate counts,97.15,51.18,97.71,46.58,40.07,38.55,17.96,38.00,56.70,25.77,32.52
Homeowner vacancy rate counts percent,0.18,0.09,0.18,0.09,0.07,0.07,0.03,0.07,0.10,0.05,0.06


### Units in structure

In [157]:
grouping = groups[1]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
current_labels = current_labels.iloc[1:]
# current_labels = current_labels.iloc[:-16]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)

UNITS IN STRUCTURE

Estimate!!UNITS IN STRUCTURE!!Total housing units!!1-unit, detached
Estimate!!UNITS IN STRUCTURE!!Total housing units!!1-unit, attached
Estimate!!UNITS IN STRUCTURE!!Total housing units!!2 units
Estimate!!UNITS IN STRUCTURE!!Total housing units!!3 or 4 units
Estimate!!UNITS IN STRUCTURE!!Total housing units!!5 to 9 units
Estimate!!UNITS IN STRUCTURE!!Total housing units!!10 to 19 units
Estimate!!UNITS IN STRUCTURE!!Total housing units!!20 or more units
Estimate!!UNITS IN STRUCTURE!!Total housing units!!Mobile home
Estimate!!UNITS IN STRUCTURE!!Total housing units!!Boat, RV, van, etc.


In [158]:
final2 = cal_district_numbers(labels_check)    
final2

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: RuntimeWarning: divide by zero encountered in double_scalars


,0,1,2,3,4,5,6,7,8,9,10
"1-unit, detached counts",147676.57,31288.53,37142.12,37583.50,33618.75,34393.06,41071.34,34854.69,29592.71,38040.00,41369.71
"1-unit, detached counts percent",0.29,0.06,0.07,0.07,0.07,0.07,0.08,0.07,0.06,0.08,0.08
"1-unit, detached moe",2404.12,1072.21,1072.98,1094.90,903.03,1057.59,1312.01,1103.70,958.50,1003.85,1027.02
"1-unit, detached moa percent",0.01,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02
"1-unit, attached counts",3823.98,1294.55,1482.10,786.77,813.48,908.83,1092.60,2582.26,1873.25,1542.99,1662.19
"1-unit, attached counts percent",0.21,0.07,0.08,0.04,0.05,0.05,0.06,0.14,0.10,0.09,0.09
"1-unit, attached moe",473.00,313.38,352.38,201.74,177.99,184.92,242.88,374.27,341.35,360.71,293.56
"1-unit, attached moa percent",0.08,0.15,0.14,0.16,0.13,0.12,0.14,0.09,0.11,0.14,0.11
2 units counts,780.47,2730.82,1716.06,1353.51,845.43,1718.09,373.79,1037.67,959.35,736.48,597.34
2 units counts percent,0.06,0.21,0.13,0.11,0.07,0.13,0.03,0.08,0.07,0.06,0.05


### Year Built

In [159]:
grouping = groups[2]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
current_labels = current_labels.iloc[1:]
# current_labels = current_labels.iloc[:-16]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)

YEAR STRUCTURE BUILT

Estimate!!YEAR STRUCTURE BUILT!!Total housing units!!Built 2020 or later
Estimate!!YEAR STRUCTURE BUILT!!Total housing units!!Built 2010 to 2019
Estimate!!YEAR STRUCTURE BUILT!!Total housing units!!Built 2000 to 2009
Estimate!!YEAR STRUCTURE BUILT!!Total housing units!!Built 1990 to 1999
Estimate!!YEAR STRUCTURE BUILT!!Total housing units!!Built 1980 to 1989
Estimate!!YEAR STRUCTURE BUILT!!Total housing units!!Built 1970 to 1979
Estimate!!YEAR STRUCTURE BUILT!!Total housing units!!Built 1960 to 1969
Estimate!!YEAR STRUCTURE BUILT!!Total housing units!!Built 1950 to 1959
Estimate!!YEAR STRUCTURE BUILT!!Total housing units!!Built 1940 to 1949
Estimate!!YEAR STRUCTURE BUILT!!Total housing units!!Built 1939 or earlier


In [160]:
final3 = cal_district_numbers(labels_check)
final3

,0,1,2,3,4,5,6,7,8,9,10
Built 2020 or later counts,1995.13,23.06,383.32,97.40,120.84,35.40,170.64,149.37,77.78,135.74,88.31
Built 2020 or later counts percent,0.61,0.01,0.12,0.03,0.04,0.01,0.05,0.05,0.02,0.04,0.03
Built 2020 or later moe,396.65,87.30,173.70,111.17,105.52,86.49,159.35,124.80,105.77,133.81,101.21
Built 2020 or later moa percent,0.12,2.30,0.28,0.69,0.53,1.49,0.57,0.51,0.83,0.60,0.70
Built 2010 to 2019 counts,52936.95,3938.05,6617.04,5121.28,4058.69,1753.59,8611.38,3617.21,14767.07,7879.29,5492.46
Built 2010 to 2019 counts percent,0.46,0.03,0.06,0.04,0.04,0.02,0.08,0.03,0.13,0.07,0.05
Built 2010 to 2019 moe,1884.67,474.92,560.16,538.45,477.90,298.37,684.53,430.40,865.80,677.80,522.21
Built 2010 to 2019 moa percent,0.02,0.07,0.05,0.06,0.07,0.10,0.05,0.07,0.04,0.05,0.06
Built 2000 to 2009 counts,50935.18,2354.67,8214.46,5216.41,9277.36,2839.46,12300.31,7481.27,15997.61,14832.76,9556.50
Built 2000 to 2009 counts percent,0.37,0.02,0.06,0.04,0.07,0.02,0.09,0.05,0.12,0.11,0.07


### Rooms

In [180]:
grouping = groups[3]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
current_labels = current_labels.iloc[1:]
current_labels = current_labels.iloc[:-7]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)

ROOMS

Estimate!!ROOMS!!Total housing units!!1 room
Estimate!!ROOMS!!Total housing units!!2 rooms
Estimate!!ROOMS!!Total housing units!!3 rooms
Estimate!!ROOMS!!Total housing units!!4 rooms
Estimate!!ROOMS!!Total housing units!!5 rooms
Estimate!!ROOMS!!Total housing units!!6 rooms
Estimate!!ROOMS!!Total housing units!!7 rooms
Estimate!!ROOMS!!Total housing units!!8 rooms
Estimate!!ROOMS!!Total housing units!!9 rooms or more
Estimate!!ROOMS!!Total housing units!!Median rooms


In [181]:
final4 = cal_district_numbers(labels_check)
final4

,0,1,2,3,4,5,6,7,8,9,10
1 room counts,2636.95,3387.01,1534.98,1686.14,800.65,1435.98,1528.63,2015.08,3676.73,1649.37,1616.48
1 room counts percent,0.12,0.15,0.07,0.08,0.04,0.07,0.07,0.09,0.17,0.08,0.07
1 room moe,465.47,509.71,304.17,325.98,253.84,289.52,352.19,353.45,536.45,465.64,384.33
1 room moa percent,0.11,0.09,0.12,0.12,0.19,0.12,0.14,0.11,0.09,0.17,0.14
2 rooms counts,4203.58,2928.19,2565.53,2367.18,1102.82,1248.58,2458.20,2661.25,6580.10,2923.77,2349.78
2 rooms counts percent,0.13,0.09,0.08,0.08,0.04,0.04,0.08,0.08,0.21,0.09,0.07
2 rooms moe,590.44,388.62,471.35,457.31,240.17,295.28,490.88,451.05,739.34,468.23,409.96
2 rooms moa percent,0.09,0.08,0.11,0.12,0.13,0.14,0.12,0.10,0.07,0.10,0.11
3 rooms counts,9216.77,9740.80,6170.45,5560.06,3339.41,5204.89,5729.25,6947.85,14029.09,9870.13,5875.29
3 rooms counts percent,0.11,0.12,0.08,0.07,0.04,0.06,0.07,0.09,0.17,0.12,0.07


### Bedrooms

In [182]:
grouping = groups[4]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
current_labels = current_labels.iloc[1:]
# current_labels = current_labels.iloc[:-16]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)

BEDROOMS

Estimate!!BEDROOMS!!Total housing units!!No bedroom
Estimate!!BEDROOMS!!Total housing units!!1 bedroom
Estimate!!BEDROOMS!!Total housing units!!2 bedrooms
Estimate!!BEDROOMS!!Total housing units!!3 bedrooms
Estimate!!BEDROOMS!!Total housing units!!4 bedrooms
Estimate!!BEDROOMS!!Total housing units!!5 or more bedrooms


In [183]:
final5 = cal_district_numbers(labels_check)
final5

,0,1,2,3,4,5,6,7,8,9,10
No bedroom counts,2916.44,3671.44,1598.19,1745.86,909.37,1485.16,1598.26,2188.76,3757.39,1776.66,1679.48
No bedroom counts percent,0.13,0.16,0.07,0.07,0.04,0.06,0.07,0.09,0.16,0.08,0.07
No bedroom moe,486.49,518.43,305.27,327.22,259.66,292.35,356.79,367.94,537.93,474.77,387.45
No bedroom moa percent,0.10,0.09,0.12,0.11,0.17,0.12,0.14,0.10,0.09,0.16,0.14
1 bedroom counts,11274.86,13418.91,8716.29,7215.44,3511.28,6751.31,8120.70,10500.16,19920.22,14169.63,8096.21
1 bedroom counts percent,0.10,0.12,0.08,0.06,0.03,0.06,0.07,0.09,0.18,0.13,0.07
1 bedroom moe,849.15,863.71,904.12,682.10,450.00,637.12,793.32,801.90,1137.13,978.74,650.30
1 bedroom moa percent,0.05,0.04,0.06,0.06,0.08,0.06,0.06,0.05,0.03,0.04,0.05
2 bedrooms counts,21181.36,17643.25,14543.70,14551.06,6900.60,16174.65,11489.65,15068.67,20639.60,14724.61,12081.84
2 bedrooms counts percent,0.13,0.11,0.09,0.09,0.04,0.10,0.07,0.09,0.13,0.09,0.07


### Tenure

In [187]:
grouping = groups[5]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
current_labels = current_labels.iloc[1:]
# current_labels = current_labels.iloc[:-2]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)

HOUSING TENURE

Estimate!!HOUSING TENURE!!Occupied housing units!!Owner-occupied
Estimate!!HOUSING TENURE!!Occupied housing units!!Renter-occupied
Estimate!!HOUSING TENURE!!Occupied housing units!!Average household size of owner-occupied unit
Estimate!!HOUSING TENURE!!Occupied housing units!!Average household size of renter-occupied unit


In [188]:
final6 = cal_district_numbers(labels_check)
final6

,0,1,2,3,4,5,6,7,8,9,10
Owner-occupied counts,134220.34,22401.87,25273.12,29062.45,25749.50,22986.08,33563.21,29316.87,26933.87,35075.54,36302.14
Owner-occupied counts percent,0.32,0.05,0.06,0.07,0.06,0.05,0.08,0.07,0.06,0.08,0.09
Owner-occupied moe,2410.04,981.96,1018.04,1065.10,916.31,951.43,1277.99,1116.46,955.98,919.42,1105.32
Owner-occupied moa percent,0.01,0.03,0.02,0.02,0.02,0.03,0.02,0.02,0.02,0.02,0.02
Renter-occupied counts,43109.46,27372.44,25949.01,20037.69,15135.03,17975.83,24399.40,27160.01,42821.70,29657.82,22620.61
Renter-occupied counts percent,0.15,0.09,0.09,0.07,0.05,0.06,0.08,0.09,0.14,0.10,0.08
Renter-occupied moe,1604.82,1146.73,1262.25,1006.22,871.02,935.41,1173.05,1112.32,1451.17,1351.58,959.39
Renter-occupied moa percent,0.02,0.03,0.03,0.03,0.03,0.03,0.03,0.02,0.02,0.03,0.03
Average household size of owner-occupied unit counts,236.19,82.43,91.20,90.45,74.82,86.13,85.67,81.78,72.96,80.73,73.62
Average household size of owner-occupied unit counts percent,0.22,0.08,0.09,0.09,0.07,0.08,0.08,0.08,0.07,0.08,0.07


17

In [192]:
with pd.ExcelWriter('sa2020_ccd_housing.xlsx') as writer:
    final0.to_excel(writer, sheet_name='Total Housing Units')
    final1.to_excel(writer, sheet_name=groups[0])
    final2.to_excel(writer, sheet_name=groups[1])
    final3.to_excel(writer, sheet_name=groups[2])
    final4.to_excel(writer, sheet_name=groups[3])
    final5.to_excel(writer, sheet_name=groups[4])
    final6.to_excel(writer, sheet_name=groups[5])